### 9.1.1. 해제 스크립트

In [1]:
from io import BytesIO
from script import Script
stream = BytesIO(bytes.fromhex('4d04ffff001d0104455468652054696d6573203033\
2f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64\
206261696c6f757420666f722062616e6b73'))
s = Script.parse(stream)
print(s.cmds[2])

b'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks'


# 연습문제 9.1. Tx 클래스의 is_coinbase 메서드를 작성하시오.

In [2]:
class Tx:
    def is_coinbase(self):
        if len(self.tx_ins) !=1:
            return False
        first_input = self.tx_ins[0]
        if first_input.prev_tx != b'\x00'*32:
            return False
        if first_input.prev_index != 0xffffffff:
            return False
        return True

### 9.1.2. BIP0034 제안서

In [3]:
from io import BytesIO
from script import Script
from helper import little_endian_to_int
stream = BytesIO(bytes.fromhex('5e03d71b07254d696e656420627920416e74506f6f\
6c20626a31312f4542312f4144362f43205914293101fabe6d6d678e2c8c34afc36896e7d94028\
24ed38e856676ee94bfdb0c6c4bcd8b2e5666a0400000000000000c7270000a5e00e00'))
script_sig = Script.parse(stream)
print(little_endian_to_int(script_sig.cmds[0]))

465879


# 연습문제 9.2. Tx클래스의 coinbase_height 메서드를 작성하시오.

In [4]:
class Tx:
    def coinbase_height(self):
        if not self.is_coinbase():
            return None
        element = self.tx_ins[0].script_sig.cmds[0]
        return little_endian_to_int(element)

# 연습문제 9.3. Block클래스의 parse 메서드를 작성하시오.

In [5]:
class Block:
    @classmethod
    def parse(cls,s):
        version = little_endian_to_int(s.read(4))
        prev_block = s.read(32)[::-1]
        merkle_root = s.read(32)[::-1]
        timestamp = little_endian_to_int(s.read(4))
        bits=s.read(4)
        nonce = s.read(4)
        return cls(version, prev_block, merkle_root,timestamp,bits, nonce)
        

# 연습문제 9.4. Block클래스의 serialize 메서드를 작성하시오.

In [6]:
class Block:
    def serialize(self):
        result = int_to_little_endian(self.version,4)
        result +=self.prev_block[::-1]
        result +=self.merkle_root[::-1]
        result +=int_to_little_endian(self.timestamp,4)
        result +=self.bits
        result += self.nonce
        return result

# 연습문제 9.5. Block클래스의 hash 메서드를 작성하시오.

In [7]:
class Block:
    def hash(self):
        s= self.serialize()
        sha = hash256(s)
        return sha[::-1]

# 연습문제 9.6. Block클래스의 bip9 메서드를 작성하시오.

In [8]:
class Block:
    def bip9(seilf):
        return self.version >>29==0b001

# 연습문제 9.7. Block클래스의 bip91 메서드를 작성하시오.

In [9]:
class Block:
    def bip91(self):
        return self.version >> 4& 1 ==1

# 연습문제 9.8. Block클래스의 bip141 메서드를 작성하시오.

In [10]:
class Block:
    def bip141(self):
        return self.version >> 1&1==1

# 연습문제 9.9. helper.py 파일의 bits_to_target 함수를 작성하시오.

In [11]:
from helper import little_endian_to_int
bits=bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient*256**(exponent-3)
difficulty = 0xffff * 256 **(0x1d-3)/target
print(difficulty)

888171856257.3206


# 연습문제 9.10. Block 클래스의 difficulty 메서드를 작성하시오.

In [12]:
class Block:
    def difficulty(self):
        lowest = 0xffff*256**(0x1d-3)
        return lowest / self.target()

# 연습문제 9.11. Block 클래스의 check_pow 메서드를 작성하시오.

In [13]:
class Block:
    def check_pow(self):
        sha = hash256(self.serialize())
        proof = little_endian_to_int(sha)
        return proof < self.target()

In [14]:
def target_to_bits(target):
    raw_bytes = target.to_bytes(32,'big')
    raw_bytes = raw_bytes.lstrip(b'\x00')
    if raw_bytes[0] > 0x7f:
        exponent = len(raw_bytes)+1
        coefficient=b'\x00'+raw_bytes[:2]
    else:
        exponent = len(raw_bytes)
        coefficient = raw_bytes[:3]
    new_bits = coefficient[::-1]+bytes([exponent])
    return new_bits

In [15]:
from block import Block
from helper import TWO_WEEKS
last_block = Block.parse(BytesIO(bytes.fromhex('00000020fdf740b0e49cf75bb3\
d5168fb3586f7613dcc5cd89675b0100000000000000002e37b144c0baced07eb7e7b64da916cd\
3121f2427005551aeb0ec6a6402ac7d7f0e4235954d801187f5da9f5')))
first_block = Block.parse(BytesIO(bytes.fromhex('000000201ecd89664fd205a37\
566e694269ed76e425803003628ab010000000000000000bfcade29d080d9aae8fd461254b0418\
05ae442749f2a40100440fc0e3d5868e55019345954d80118a1721b2e')))

time_differential = last_block.timestamp - first_block.timestamp
if time_differential > TWO_WEEKS * 4:
    time_differential = TWO_WEEKS * 4
if time_differential < TWO_WEEKS // 4:
    time_differential = TWO_WEEKS // 4
new_target = last_block.target() * time_differential // TWO_WEEKS
print('{:x}'.format(new_target).zfill(64))

IndentationError: expected an indented block (block.py, line 37)